In [6]:
# week 3 assignment Segmenting and Clustering Neighborhoods in Toronto
import json 
import pandas as pd
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium 
import folium 
!pip install lxml
import lxml
import numpy as np
print('Libraries imported.')
%matplotlib inline

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [7]:
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1004901187')[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
# first remove cells where borough is not assigned
# then combine cells where postal code or borough are the same
# lastly find where neighbourhood is not assigned and replace with borough.

df_filtered = df[df['Borough'] != 'Not assigned'] 
df_filtered.reset_index(drop=True, inplace=True)
df_filtered = df_filtered.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))

NAneigh = df_filtered['Neighbourhood'] == 'Not assigned'
df_filtered.loc[NAneigh, 'Neighbourhood'] = df_filtered.loc[NAneigh, 'Borough']

df_filtered.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df_filtered.shape
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [10]:
df = pd.merge(geocsv_data, df, on='Postal Code')
df = df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
